In [1]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, get_linear_schedule_with_warmup, AdamW
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
import datasets 
# from datasets import Dataset, DatasetDict
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm

import logging
logging.getLogger().setLevel(logging.CRITICAL)

import warnings
warnings.filterwarnings('ignore')

device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

# Data Preprocessing

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [32]:
import re

def clean_text(line):
    pattern = re.compile(r'[^a-zA-Z0-9\s,.!?[\]]')
    line = pattern.sub('', line)
    line = re.sub(r'\s+', ' ', line).strip()
    return line

# lyrics_train_df = pd.read_csv('./data/lyrics_train.csv')  # No need for index_col=False, as it's False by default
# lyrics_train_df['Lyric'] = lyrics_train_df['Lyric'].str.replace("\r\n", " ").str.replace("\r", " ").str.replace("\n", " ")
# lyrics_train_df['Lyric'] = lyrics_train_df['Lyric'].apply(clean_text)
# lyrics_train_df.to_csv('./cleaned_train_lyrics.csv', index=False)
lyrics_train_df = pd.read_csv('./data/cleaned_train_lyrics.csv')

lyrics_train_df

,Unnamed: 0,Lyric,genre
0,0,"See me, ancient one! Dismal Tuat, Nergal unsaf...",Metal
1,1,Feels like Im covered in lies so turn off the ...,Metal
2,2,"Works of art, painted black Magniloquent, blee...",Metal
3,3,Into the cage like an animal You must survive ...,Metal
4,4,Paralysed in pleasure I hear you call Lost my ...,Metal
...,...,...,...
499995,499995,[Verse 1] I dont want to tell you that its ove...,country
499996,499996,I get to thinking sometimes I dont know why I ...,country
499997,499997,When I was A little boy around the table athom...,country
499998,499998,[Verse 1] Its a junked out joint off a backroa...,country


In [12]:
import re

def clean_text(line):
    pattern = re.compile(r'[^a-zA-Z0-9\s,.!?[\]]')
    line = pattern.sub('', line)
    line = re.sub(r'\s+', ' ', line).strip()
    return line

# lyrics_test_df = pd.read_csv('./data/lyrics_test.csv')  # No need for index_col=False, as it's False by default
# lyrics_test_df['Lyric'] = lyrics_test_df['Lyric'].str.replace("\r\n", " ").str.replace("\r", " ").str.replace("\n", " ")
# lyrics_test_df['Lyric'] = lyrics_test_df['Lyric'].apply(clean_text)
# lyrics_test_df.to_csv('./data/cleaned_test_lyrics.csv', index=False)
lyrics_test_df = pd.read_csv('./data/cleaned_test_lyrics.csv')

lyrics_test_df

,Unnamed: 0,Lyric,genre
0,0,can you hear me call your name Im not far away...,Metal
1,1,You say you are so clever You beleive that you...,Metal
2,2,Walking across these misery plains When all fo...,Metal
3,3,Fuck you you bitch get out of my head Twisting...,Metal
4,4,Crashing forth upon the soil Filthy waves gave...,Metal
...,...,...,...
49995,49995,[Verse 1] When the sun sinks down and dreams s...,country
49996,49996,I watched from the window as she slipped from ...,country
49997,49997,"Look around, it is never far See who the wound...",country
49998,49998,We started arguing on the onramp Of Interstate...,country


In [4]:
class LyricsDataset(Dataset):
    def __init__(self, lyrics_df):
        super().__init__()


        self.lyric_list = []
        self.end_of_text_token = "<|endoflyric|>"
        
        for lyric, genre in tqdm(zip(lyrics_df['Lyric'],lyrics_df['genre'] ), total=len(lyrics_df['genre'])):
            lyric_str = f"LYRIC[{genre.lower()}]:{lyric}{self.end_of_text_token}"
#             lyric_str = f"LYRIC:{lyric}{self.end_of_text_token}"
            self.lyric_list.append(lyric_str)
        
    def __len__(self):
        return len(self.lyric_list)

    def __getitem__(self, item):
        return self.lyric_list[item]

In [5]:
# train_data = LyricsDataset(lyrics_train_df)
test_data = LyricsDataset(lyrics_test_df[lyrics_test_df['genre'] == 'Metal'])

# train_loader = DataLoader(train_data, batch_size=1, shuffle=True)
test_loader = DataLoader(test_data, batch_size=1, shuffle=True)

  0%|          | 0/10000 [00:00<?, ?it/s]

# Training Pipelines

In [50]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.to(device)

tokenizer.add_special_tokens({
    'pad_token':'<|pad|>'
                             })

1

In [51]:
def calc_perp(model, tokenizer, test_loader):
    model.eval()
    nlls = []
    for lyric in tqdm(test_loader):
        lyric_tens = tokenizer(lyric, padding=True, truncation= True, return_tensors='pt')['input_ids'].to(device)
        target_tens = lyric_tens.clone()
        with torch.no_grad():
            outputs = model(lyric_tens, labels=target_tens)
            neg_log_likelihood = outputs.loss
            nlls.append(neg_log_likelihood)
    ppl = torch.exp(torch.stack(nlls).mean())
    print("Evaluations:", ppl.item())
    model.train()

In [8]:
BATCH_SIZE = 32
EPOCHS = 5
LEARNING_RATE = 5e-6
WARMUP_STEPS = 80000
EVAL_STEPS = 100000
PRINT_STEPS = 100


optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=WARMUP_STEPS, num_training_steps=-1)
proc_seq_count = 0


proc_seq_count = 0
sum_loss = 0.0
batch_count = 0
steps = 0

In [9]:
model.train()
for epoch in range(EPOCHS):
    
    print(f"EPOCH {epoch} started" + '=' * 30)
    
    for idx,lyric in enumerate(tqdm(train_loader)):
        lyric_tens = tokenizer(lyric, padding=True, return_tensors='pt', truncation= True)['input_ids'].to(device)
        output = model(lyric_tens, labels = lyric_tens.clone())
        loss = output['loss']  / BATCH_SIZE
        loss.backward()
        sum_loss = sum_loss + output['loss'].detach().data
        
        proc_seq_count = proc_seq_count + 1
        steps += 1
        if proc_seq_count == BATCH_SIZE:
            proc_seq_count = 0    
            batch_count += 1
            optimizer.step()
            scheduler.step() 
            optimizer.zero_grad()
            model.zero_grad()

        if batch_count == PRINT_STEPS:
            print(f"sum loss {sum_loss}")
            batch_count = 0
            sum_loss = 0.0
        if steps > EVAL_STEPS:
            steps = 0
            calc_perp(model, tokenizer, test_loader)
            model.push_to_hub("multi-genre-mdeium")
            tokenizer.push_to_hub("multi-genre-mdeium")

EPOCH 0 started==============================


  0%|          | 0/500000 [00:00<?, ?it/s]

KeyboardInterrupt: 

# Evaluations Perplexity

In [6]:
import random 


device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'
    
    
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")
model = AutoModelForCausalLM.from_pretrained("openai-community/gpt2")

tokenizer.add_special_tokens({
    'pad_token':'<|pad|>'
                             })
model.to(device)

def evaluate(model, tokenizer, test_loader):
    model.eval()
    nlls = []
    EVAL_STEPS = 1000
    steps = 0
    for lyric in tqdm(test_loader):
        lyric_tens = tokenizer(lyric, padding=True, truncation= True, return_tensors='pt')['input_ids'].to(device)
        target_tens = lyric_tens.clone()
        with torch.no_grad():
            outputs = model(lyric_tens, labels=target_tens)
            neg_log_likelihood = outputs.loss
            nlls.append(neg_log_likelihood)
        if EVAL_STEPS == steps:
            steps = 0
            ppl = torch.exp(torch.stack(nlls).mean())
            print("Evaluations:", ppl.item())
        steps += 1

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [7]:
for genre in lyrics_test_df['genre'].unique():
    # train_data = LyricsDataset(lyrics_train_df)
    test_data = LyricsDataset(lyrics_test_df[lyrics_test_df['genre'] == genre])

    # train_loader = DataLoader(train_data, batch_size=1, shuffle=True)
    test_loader = DataLoader(test_data, batch_size=1, shuffle=True)
    evaluate(model, tokenizer, test_loader)

  0%|          | 0/10000 [00:00<?, ?it/s]

Evaluations: 89.34149169921875
Evaluations: 91.60322570800781
Evaluations: 90.94165802001953


KeyboardInterrupt: 

# Inference

In [11]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("D3STRON/multi_genre_music_generator")
model = AutoModelForCausalLM.from_pretrained("D3STRON/multi_genre_music_generator")
model.to(device)

def choose_from_top(probs, cur_ids, n=7, no_rep=2):
    ind = np.argpartition(probs, -n)[-n:]
    top_prob = probs[ind]
    top_prob /= np.sum(top_prob)  # Normalize

    cur_ids_list = cur_ids.squeeze().cpu().numpy().tolist()
    for i in range(200):
        if i < 4:
            choice = np.random.choice(ind, 1)[0]
        else:
            choice = np.random.choice(ind, 1, p=top_prob)[0]
        n_gram = cur_ids_list[-no_rep + 1 :] + [choice]
        if any(cur_ids_list[i : i + no_rep] == n_gram for i in range(len(cur_ids_list) - no_rep + 1)):
            continue
        return choice
    return np.random.choice(ind)

In [24]:
with torch.no_grad():
    cur_ids = tokenizer("LYRIC[pop]:A bundle of joy", padding=True, return_tensors='pt', truncation= True)['input_ids'].to(device)
    for i in range(100):
        outputs = model(cur_ids, labels=cur_ids)
        logits = outputs['logits']
        softmax_logits = torch.softmax(logits[0,-1], dim=0) #Take the first(from only one in this case) batch and the last predicted embedding
        if i == 3:
            n = 30
        else:
            n = 5
        next_token_id = choose_from_top(softmax_logits.to('cpu').numpy(), cur_ids, n=n) #Randomly(from the topN probability distribution) select the next word
        cur_ids = torch.cat([cur_ids, torch.ones((1,1)).long().to(device) * next_token_id], dim = 1) # Add the last word to the running sequence

        if next_token_id in tokenizer.encode('<|endoflyric|>'):
            joke_finished = True
            break

    output_list = list(cur_ids.squeeze().to('cpu').numpy())
    output_text = tokenizer.decode(output_list)
    print(output_text)

LYRIC[pop]:A bundle of joy A promise that could not be fulfilled The world is filled with love and happiness And all that I want for you is a place in the sky But you cant find me on your knees And the sun wont set on my heart But its all right for me And youre a part in a beautiful life And I dont have no shame But Ive never known the way you feel Youre the love that you never knew you needed I know the world has been full for years but youve always been here For


# Evaluation SET

In [34]:
from datasets import load_metric
from transformers import pipeline
import tqdm.auto as tqdm
import torch
import pandas as pd

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
lyrics_test_df = pd.read_csv('./data/cleaned_test_lyrics.csv')

pipe = pipeline("text-generation", model="D3STRON/multi_genre_music_generator", device = device)
# pipe = pipeline("text-generation", model="D3STRON/multi-genre", device = device)



# Generate text


generations = {'pop':[],
         'rock':[],
         'metal':[],
         'country':[],
         'rap':[]}


for genre in lyrics_test_df['genre'].unique():
    for batch in tqdm.tqdm(range(50)):
        start_prompt = f"LYRIC[{genre}]:"
        generated_texts = pipe(start_prompt, max_length=200, truncation= True, num_return_sequences=20)
        generated_texts = [generated_text['generated_text'] for generated_text in generated_texts]
        generations[genre.lower()] += generated_texts

  0%|          | 0/50 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
C:\Users\GHOSH\anaconda3\lib\site-packages\transformers\pipelines\base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end g

  0%|          | 0/50 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

  0%|          | 0/50 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

KeyboardInterrupt: 

In [5]:
for key in generations:
    for i in tqdm.tqdm(range(len(generations[key]))):
        generations[key][i] = generations[key][i].split("<|endoflyric|>", 1)[0]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

In [27]:
import json

# File path to save the dictionary
file_path = "./results/gpt2_med_f.json"

In [28]:
import pandas as pd

with open(file_path, "r") as json_file:
    load_dict = json.load(json_file)
lyrics_test_df = pd.read_csv('./data/cleaned_test_lyrics.csv')

# ROGUE Evaluation

In [56]:
from nltk import ngrams
from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import SmoothingFunction
import re
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('wordnet')

def preprocess_text(text):
    # Remove text within square brackets
    text = re.sub(r'\[.*?\]', '', text)
    
    # Tokenize the text
    tokens = word_tokenize(text)
    tokens = [word for word in tqdm.tqdm(tokens) if word.isalnum()]
    # Join tokens back into a string
    preprocessed_text = " ".join(tokens)
    
    return preprocessed_text


def rouge_precision_recall(reference, candidate, n):
    reference_tokens = word_tokenize(reference)
    candidate_tokens = word_tokenize(candidate)
    
    reference_ngrams = list(ngrams(reference_tokens, n))
    candidate_ngrams = list(ngrams(candidate_tokens, n))
    
    # Calculate precision
    overlapping_ngrams = set(reference_ngrams) & set(candidate_ngrams)
    precision = len(overlapping_ngrams) / len(candidate_ngrams) if len(candidate_ngrams) > 0 else 0
    
    # Calculate recall
    recall = len(overlapping_ngrams)*10 / len(reference_ngrams) if len(reference_ngrams) > 0 else 0
    
    f1_score = 2*precision*recall/(precision + recall)
    
    return precision, recall, f1_score

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\GHOSH\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\GHOSH\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [51]:
file_path = "./results/gpt2_med_f.json"

with open(file_path, "r") as json_file:
    load_dict = json.load(json_file)

for genre in lyrics_test_df['genre'].unique():
    reference_text = lyrics_test_df.loc[lyrics_test_df['genre'] == genre, 'Lyric'].tolist()
    reference_text = " ".join(reference_text)

    generated_text = load_dict[genre.lower()]
    generated_text = " ".join(generated_text)
    reference_text = preprocess_text(reference_text)
    generated_text = preprocess_text(generated_text)
    
    print(f"For {genre}:")
    for i in range(1,4):
        precision, recall, f1_score = rouge_precision_recall(reference_text, generated_text, i)
        print(f"Rogue{i} Precision: {precision}\nRogue{i} Recall: {recall}\nRogue{i} F1: {f1_score}")

  0%|          | 0/1751623 [00:00<?, ?it/s]

  0%|          | 0/125656 [00:00<?, ?it/s]

For Metal:
Rogue1 Precision: 0.028698498663193253
Rogue1 Recall: 0.020355321618944003
Rogue1 F1: 0.023817397581278914
Rogue2 Precision: 0.23104674579030807
Rogue2 Recall: 0.16387593761263355
Rogue2 F1: 0.19174893562699064
Rogue3 Precision: 0.24227466707799886
Rogue3 Recall: 0.1718382658630691
Rogue3 F1: 0.20106620847391085


  0%|          | 0/2142944 [00:00<?, ?it/s]

  0%|          | 0/150238 [00:00<?, ?it/s]

...

  0%|          | 0/150990 [00:00<?, ?it/s]

For pop:
Rogue1 Precision: 0.021643306079314757
Rogue1 Recall: 0.015220275969565315
Rogue1 F1: 0.01787222364794303
Rogue2 Precision: 0.20136131988236194
Rogue2 Recall: 0.1416029065705372
Rogue2 F1: 0.1662759318143522
Rogue3 Precision: 0.25191545574636726
Rogue3 Recall: 0.17715284242476395
Rogue3 F1: 0.2080206774838414


  0%|          | 0/2259320 [00:00<?, ?it/s]

  0%|          | 0/161635 [00:00<?, ?it/s]

For country:
Rogue1 Precision: 0.020496196515517442
Rogue1 Recall: 0.01462782014141148
Rogue1 F1: 0.01707177622311424
Rogue2 Precision: 0.1995130991818258
Rogue2 Recall: 0.14238857573172548
Rogue2 F1: 0.16617868888478318
Rogue3 Precision: 0.25571498682647104
Rogue3 Recall: 0.18249766341788043
Rogue3 F1: 0.21298968695104017


In [52]:
file_path = "./results/gpt2_small_f.json"

with open(file_path, "r") as json_file:
    load_dict = json.load(json_file)

for genre in lyrics_test_df['genre'].unique():
    reference_text = lyrics_test_df.loc[lyrics_test_df['genre'] == genre, 'Lyric'].tolist()
    reference_text = " ".join(reference_text)

    generated_text = load_dict[genre.lower()]
    generated_text = " ".join(generated_text)
    reference_text = preprocess_text(reference_text)
    generated_text = preprocess_text(generated_text)
    
    print(f"For {genre}:")
    for i in range(1,4):
        precision, recall, f1_score = rouge_precision_recall(reference_text, generated_text, i)
        print(f"Rogue{i} Precision: {precision}\nRogue{i} Recall: {recall}\nRogue{i} F1: {f1_score}")

  0%|          | 0/1751623 [00:00<?, ?it/s]

  0%|          | 0/123781 [00:00<?, ?it/s]

For Metal:
Rogue1 Precision: 0.02694219878085686
Rogue1 Recall: 0.018939140688154527
Rogue1 F1: 0.022242685111820176
Rogue2 Precision: 0.2186781261348505
Rogue2 Recall: 0.15371954373418573
Rogue2 F1: 0.18053336255255087
Rogue3 Precision: 0.22615064027738147
Rogue3 Recall: 0.1589710612495958
Rogue3 F1: 0.18670153951141372


  0%|          | 0/2142944 [00:00<?, ?it/s]

  0%|          | 0/145545 [00:00<?, ?it/s]

...

  0%|          | 0/146350 [00:00<?, ?it/s]

For pop:
Rogue1 Precision: 0.020426560968455094
Rogue1 Recall: 0.0139588460947989
Rogue1 F1: 0.016584431894621766
Rogue2 Precision: 0.19471552859023525
Rogue2 Recall: 0.13306135979273426
Rogue2 F1: 0.15808993205589233
Rogue3 Precision: 0.23768128385098056
Rogue3 Recall: 0.16242147835814474
Rogue3 F1: 0.19297315163728015


  0%|          | 0/2259320 [00:00<?, ?it/s]

  0%|          | 0/157169 [00:00<?, ?it/s]

For country:
Rogue1 Precision: 0.019015570865357812
Rogue1 Recall: 0.013203750694752327
Rogue1 F1: 0.015585483794632825
Rogue2 Precision: 0.18509275545083465
Rogue2 Recall: 0.12852117463201154
Rogue2 F1: 0.15170460279065315
Rogue3 Precision: 0.23631391706047894
Rogue3 Recall: 0.16408613369858366
Rogue3 F1: 0.19368547489498092


In [53]:
file_path = "./results/gpt2_small_nf.json"

with open(file_path, "r") as json_file:
    load_dict = json.load(json_file)

for genre in lyrics_test_df['genre'].unique():
    reference_text = lyrics_test_df.loc[lyrics_test_df['genre'] == genre, 'Lyric'].tolist()
    reference_text = " ".join(reference_text)

    generated_text = load_dict[genre.lower()]
    generated_text = " ".join(generated_text)
    reference_text = preprocess_text(reference_text)
    generated_text = preprocess_text(generated_text)
    
    print(f"For {genre}:")
    for i in range(1,4):
        precision, recall, f1_score = rouge_precision_recall(reference_text, generated_text, i)
        print(f"Rogue{i} Precision: {precision}\nRogue{i} Recall: {recall}\nRogue{i} F1: {f1_score}")

  0%|          | 0/1751623 [00:00<?, ?it/s]

  0%|          | 0/88735 [00:00<?, ?it/s]

For Metal:
Rogue1 Precision: 0.0965940445445114
Rogue1 Recall: 0.026597458167960275
Rogue1 F1: 0.041709955678406324
Rogue2 Precision: 0.1476314511500596
Rogue2 Recall: 0.04064988764755186
Rogue2 F1: 0.06374717686648503
Rogue3 Precision: 0.06037394318006225
Rogue3 Recall: 0.016623431562517475
Rogue3 F1: 0.026068995624030752


  0%|          | 0/2142944 [00:00<?, ?it/s]

  0%|          | 0/102772 [00:00<?, ?it/s]

...

  0%|          | 0/103205 [00:00<?, ?it/s]

For pop:
Rogue1 Precision: 0.09193713547927519
Rogue1 Recall: 0.02019265652242363
Rogue1 F1: 0.03311260932973825
Rogue2 Precision: 0.16522339217737805
Rogue2 Recall: 0.03628812832561726
Rogue2 F1: 0.059506748227204816
Rogue3 Precision: 0.07459928762243989
Rogue3 Recall: 0.0163839366037972
Rogue3 F1: 0.026867150719023004


  0%|          | 0/2259320 [00:00<?, ?it/s]

  0%|          | 0/110248 [00:00<?, ?it/s]

For country:
Rogue1 Precision: 0.07484121646865426
Rogue1 Recall: 0.02308006404164735
Rogue1 F1: 0.03528017730261628
Rogue2 Precision: 0.13509878351316615
Rogue2 Recall: 0.04166211563357063
Rogue2 F1: 0.06368491185381213
Rogue3 Precision: 0.062304783823771165
Rogue3 Recall: 0.019213433641989533
Rogue3 F1: 0.029369848036221637


In [57]:
file_path = "./results/gpt2_med_nf.json"

with open(file_path, "r") as json_file:
    load_dict = json.load(json_file)

for genre in lyrics_test_df['genre'].unique():
    reference_text = lyrics_test_df.loc[lyrics_test_df['genre'] == genre, 'Lyric'].tolist()
    reference_text = " ".join(reference_text)

    generated_text = load_dict[genre.lower()]
    generated_text = " ".join(generated_text)
    reference_text = preprocess_text(reference_text)
    generated_text = preprocess_text(generated_text)
    
    print(f"For {genre}:")
    for i in range(1,4):
        precision, recall, f1_score = rouge_precision_recall(reference_text, generated_text, i)
        print(f"Rogue{i} Precision: {precision}\nRogue{i} Recall: {recall}\nRogue{i} F1: {f1_score}")

  0%|          | 0/1751623 [00:00<?, ?it/s]

  0%|          | 0/111938 [00:00<?, ?it/s]

For Metal:
Rogue1 Precision: 0.08531188776698496
Rogue1 Recall: 0.037847891227579665
Rogue1 F1: 0.052433920797581655
Rogue2 Precision: 0.13331963282641457
Rogue2 Recall: 0.05914534340585035
Rogue2 F1: 0.08193943252038065
Rogue3 Precision: 0.05262436805546041
Rogue3 Recall: 0.023345740633136974
Rogue3 F1: 0.03234311148984615


  0%|          | 0/2142944 [00:00<?, ?it/s]

  0%|          | 0/120454 [00:00<?, ?it/s]

...

  0%|          | 0/110564 [00:00<?, ?it/s]

For pop:
Rogue1 Precision: 0.07969284700776229
Rogue1 Recall: 0.023341341946259177
Rogue1 F1: 0.03610719920373567
Rogue2 Precision: 0.17566439636752137
Rogue2 Recall: 0.05144974055112778
Rogue2 F1: 0.07958894800473616
Rogue3 Precision: 0.08585546633724521
Rogue3 Recall: 0.025145504611557446
Rogue3 F1: 0.0388983809106744


  0%|          | 0/2259320 [00:00<?, ?it/s]

  0%|          | 0/121828 [00:00<?, ?it/s]

For country:
Rogue1 Precision: 0.059396000323860415
Rogue1 Recall: 0.01690448780047174
Rogue1 F1: 0.02631854625190322
Rogue2 Precision: 0.11824335265731774
Rogue2 Recall: 0.033652297384550084
Rogue2 F1: 0.05239334327577504
Rogue3 Precision: 0.05387592505627257
Rogue3 Recall: 0.015332956039073921
Rogue3 F1: 0.023871999586706587


In [87]:
(0.18 - 0.063)*100/(0.063)

185.7142857142857